# Movie Recommendation System


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [14]:
print(credits.columns)
print(movies.columns)

Index(['version https://git-lfs.github.com/spec/v1'], dtype='object')
Index(['version https://git-lfs.github.com/spec/v1'], dtype='object')


In [15]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

# EDA:
1. Removing unnecessary columns 
2. Removing rows if any of the field is NaN

In [16]:
movies=movies.merge(credits, on = 'title')

KeyError: 'title'

In [11]:
movies = movies[['id','title','genres','keywords','overview','cast','crew']]

KeyError: "None of [Index(['id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew'], dtype='object')] are in the [columns]"

In [ ]:
movies.dropna(inplace=True)

In [ ]:
#remove duplicate data
movies.duplicated().sum()
#No duplicated columns


In [ ]:
import ast

In [ ]:
def convertG(obj):
    lst = ast.literal_eval(obj)
    L=[]
    for x in lst:
        L.append(x["name"])
    return L

In [ ]:
movies['genres']=movies['genres'].apply(convertG)

In [ ]:
movies['keywords']=movies['keywords'].apply(convertG)

In [ ]:
def convertC(obj):
    lst = ast.literal_eval(obj)
    count=0
    L=[]
    for x in lst:
        if(count != 5):
            L.append(x['name'])
            count+=1
        else:
            break
    return L

In [ ]:
movies['cast']=movies['cast'].apply(convertC)

In [ ]:
def convertCrew(obj):
    lst = ast.literal_eval(obj)
    L=[]
    for x in lst:
        if(x['job']=="Director" or x['job']=="Producer"):
            L.append(x['name'])
    return L

In [ ]:
def removeDuplicates(obj):
    return list(set(obj))

In [ ]:
movies['crew'] = movies['crew'].apply(convertCrew).apply(removeDuplicates)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['cast']=movies['cast'].apply(lambda x: [i.replace(' ','')for i in x])
movies['genres']=movies['genres'].apply(lambda x: [i.replace(' ','')for i in x])
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(' ','')for i in x])
movies['crew']=movies['crew'].apply(lambda x: [i.replace(' ','')for i in x])

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew']+movies['keywords']

In [ ]:
movies.drop(['overview','genres','cast','crew','keywords'],axis=1,inplace=True)

In [ ]:
movies['tags']=movies['tags'].apply(lambda x: " ".join(x))

In [ ]:
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    filtered=[]
    for i in text.split():
        filtered.append(ps.stem(i))
    return " ".join(filtered)


In [ ]:
movies['tags'] = movies['tags'].apply(stem)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(movies['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_idx = movies[movies['title']==movie].index[0]
    distances = similarity[movie_idx]
    recommended_lst = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in recommended_lst:
        print(movies.iloc[i[0]].title)

In [ ]:
recommend('Titanic')

# Now for Deployment:

In [ ]:
import pickle

In [ ]:
pickle.dump(movies,open('movies.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
movies.head()